In [ ]:
sql = """ 
    SELECT
        mo.product_family,
        mo.product_model,
        mo.product_variant,
        mo.item_description,
        mo.order_quantity,
        mo.product_length,
        mo.reference_number,
        mol.time_out,
        mol.facility_id
    FROM manufacturing_orders mo
    JOIN manufacturing_order_logs mol ON mol.order_number = mo.order_number
    WHERE mo.order_status = '55'
        AND mo.order_release_code = 5
        AND mol.time_out >= '2023-01-01'::date
        AND mol.time_out <= '2023-06-30'::date
        AND mo.product_model IS NOT NULL
        AND mo.product_family IS NOT NULL
        AND mo.product_length IS NOT NULL
        AND mo.reference_number LIKE 'HK%'
        AND mo.item_description LIKE '%RAIL%'
        AND mo.product_family != 'TS'
    ORDER BY mo.order_number;
    """
df = pd.DataFrame(call_db_json(sql))

In [ ]:
df = df[df['item_description'].apply(lambda x: x.find("K") != x.find("K)"))]
rename = {'product_variant':'total'}
df['total'] = df['order_quantity'] * df['product_length'] / 1000
df['total'] = df['total'].astype(int)
df['type'] = df['product_family'] + df['product_model']
df = df.rename(columns=rename)

In [ ]:
# df.head()
grouped = df.groupby(['type', 'facility_id'])
summed = grouped['total'].sum()
summed = summed.reset_index() 
df = pd.DataFrame(summed)
df = df.pivot_table(index=['type'], columns='facility_id', values=['total']).reset_index()
df.fillna(0)
